In [2]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M
from scipy.optimize import fsolve

# Simple, conceptual model of marginal sea

In [21]:
############ Initilisation #################
############################################

#### Constants #####
L_x = 1e6          # [m] zonal extent 
L_y = 2e6          # [m] meridional extent 
A = L_x*L_y        # [m^2] surface area of interior 
P = 2*L_x+2*L_y    # [m] Perimeter 

g = 9.81           # [m/s^2] gravitational acceleration 
C_p = 3.994e3        # [J/(kg K)] specific heat capacity of sea water 
c = 0.006          # [] bottom slope coefficient
s = 0.02           # [] average bottom slope around the perimeter of the marginal sea

#### Parameters ####
H = 1000           # [m] sill height 
f_0 = 1.2e-4       # [s^-1] Coriolis parameter 
Gamma = 20         # [W/(m^2 C)] Restoring strength 
alpha_T = 0.2      # [kg m^-3 C^-1] thermal expansion coefficient; source: Ypma[2020]
alpha_S = 0.8      # [kg m^-3] Haline expansion coefficient 
T_1 = 8.87         # [Celsius] ingoing temperature 
T_A = 2.8          # [Celsius] Atmospheric temperature 
ro = 1000          # [kg m^-3] density sea water 
S = 35             # [ppt] net evaporation minus precipitation
S_1 = 35.2         # [g kg^-1] incoming salinity; source Lambert                          ! is ppt = g kg^-1 ???
E = -2e-8          # [m/s] a representative ocean salinity; source: Spall[2015]

#### functions ####
L = H/s           # [m] width of sloping topography of the boundary current 
#L = 75e3

In [22]:
############# Computation ##################
############################################

# Main equations

def main(x): # function with seven unkowns:  T_0,  T_2,  S_0,  S_2,  V_1, T_eddy, S_eddy 
             #                              x[0], x[1], x[2], x[3], x[4],   x[5],   x[6]
    return [P*H*x[5]-A*Gamma*(x[0]-T_A)/(ro*C_p),
            P*H*x[6]+A*E*S,
            (T_1-x[1])*x[4]*H*L-(x[0] - T_A + (P*L/A)*(T_1-T_A))*A*Gamma/(ro*C_p),
            (S_1-x[3])*x[4]*H*L+E*A*S+P*L*E*S,
            g*H/(2*ro*f_0*L)*(alpha_T*(T_1-x[0])-alpha_S*(S_1-x[2]))-x[4],
            c*x[4]*(T_1-x[0])-x[5],
            c*x[4]*(S_1-x[2])-x[6]]

[T_0,T_2,S_0,S_2,V_1,T_eddy,S_eddy] = fsolve(main, [1,1,1,1,1,1,1])

In [23]:
[T_0,T_2,S_0,S_2,V_1,T_eddy,S_eddy]

[6.229528774722218,
 6.464130350471608,
 35.092372539544435,
 35.110884462047224,
 0.361328686194978,
 0.005724467993193487,
 0.00023333333333333333]

In [48]:
#### Follow up computations ####

#Atmospheric heat flux/meridional heat transport at the sill [W/m^2]
Q_star = A*(T_0 - T_A)*Gamma + P*L*Gamma*(T_1-T_A)

#Fraction of outflowing waters that include inflowing water [-]
Delta = (T_2-T_0)/(T_1-T_0)

#Downwelling/Medidional overturning strength at the sill [m^2/s]
W = alpha_T*(g*H**2*(T_1-T_0)*(1-Delta))/(4*ro_0*f_0)


In [50]:
print('V_1=', "{:.2e}".format(V_1), ', Q_star=', "{:.2e}".format(Q_star), ', W=', "{:.2e}".format(W))

V_1= 2.50e-02 , Q_star= 3.64e+13 , W= -2.69e+07
